In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_split(n=256, stride=1):
    m = stride*4
    ins = []
    fins = []
    ins.append(0)
    ins.append(0)
    fins.append(m)
    fins.append(2*m)
    for i in range(int(np.log2(n//m))):
        iin = 2**(i)*m
        l = 3*(2**i)*m
        ifin = iin + l
        ins.append(iin)
        fins.append(np.minimum(ifin, n))
    return ins, fins

def split_sigs(x,ins,fins):
    xs = []
    for i,j in zip(ins,fins):
        xs.append(x[:,:,i:j,:])
    return xs
def merge_sigs(x, ins, fins):
    bs, ch,_, ts = x[0].shape
    m = np.zeros([bs, ch, np.max(fins), ts])
    for d,i,j in zip(x, ins,fins):
        m[:,:,i:j,:] += d
    return m/2

n = 256
ts = 80
bs = 8
ch = 15
a = np.random.randn(bs, ch, n, ts) # Bs-Channels-Freq-Time
ins, fins = get_split(n=n, stride=1)

np.testing.assert_almost_equal(merge_sigs(split_sigs(a, ins, fins), ins, fins),a)

In [ ]:
total_sum = np.zeros([n])
for i,j in zip(ins,fins):
    total_sum[i:j] += 1
    win = np.zeros([n])
    win[i:j]+=1
    plt.plot(win)
plt.plot(total_sum)

In [ ]:
def np_downsample_1d(x, scaling):
    """1 dimensional numpy downscaling"""

    bs, ch, n, ts = x.shape
    nx = n // scaling
    dsx = np.zeros([bs, ch, nx, ts])
    for i in range(scaling):
        dsx += x[:,:, i:scaling * nx + i:scaling,:]
    dsx /= scaling
    return dsx

In [ ]:
def np_upsample_1d(x, scaling):
    return np.repeat(x,scaling, axis=2)

In [ ]:
stride = 2

a = np.random.randn(bs, ch, n, ts) # Bs-Channels-Freq-Time

# Smoothing the parts
ins_1, fins_1 = get_split(n=n, stride=stride)
xs = split_sigs(a, ins_1, fins_1)

xs_ds = [np_downsample_1d(el, stride) for el in xs]

ins_2, fins_2 = get_split(n=n//stride, stride=1)
a_rebuid = merge_sigs(xs_ds, ins_2, fins_2)

# Compare with a direct smoothing
a_small = np_downsample_1d(a, stride)
np.testing.assert_almost_equal(a_rebuid,a_small)

In [ ]:
[b-a for a,b in zip(ins,fins)]